# Trial Onset ~ all frequencies: SLCH002

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra


In [5]:
## Prep paths ##

subject = 'SLCH002'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

# load preproc functions
%run ../../scripts/preproc_functions.py

In [6]:
## Load Neural Data

# load
trial_onset_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(trial_onset_epochs.get_annotations_per_epoch()) if  x]

# only good epochs
trial_onset_epochs = trial_onset_epochs[good_epochs]

Reading /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg.fif ...


/tmp/ipykernel_134830/460025053.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  trial_onset_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif")


    Found the data of interest:
        t =   -4000.00 ...   12000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg-1.fif ...
    Found the data of interest:
        t =   -4000.00 ...   12000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg-2.fif ...
    Found the data of interest:
        t =   -4000.00 ...   12000.00 ms
        0 CTF compensation matrices available
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated


In [7]:
## Dictionary of electrode locations ##

# Pull mapping ROI to elecs
%run ../../scripts/roi.py
ROIs = ROIs[subject]

## prep lists

# limbic ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []
sgACC_list = []
sgACC_names = [] 
sgACC_indices = []
dACC_list = []
dACC_names = [] 
dACC_indices = []
sfg_list = []
sfg_names = [] 
sfg_indices = []
mfg_list = []
mfg_names = [] 
mfg_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in trial_onset_epochs.info['ch_names']]
bidx = len(trial_onset_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
        sgACC_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        sgACC_names.append(pairs_name[ix])
        sgACC_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
        dACC_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        dACC_names.append(pairs_name[ix])
        dACC_indices.append(ix)        
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
        sfg_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        sfg_names.append(pairs_name[ix])
        sfg_indices.append(ix)   
    if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
        mfg_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        mfg_names.append(pairs_name[ix])
        mfg_indices.append(ix)     
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(trial_onset_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

In [8]:
# Print and check frequencies
print(freqs)
print(n_cycles)
print(time_bin)
print(band_width)


[  1.           1.06548039   1.13524845   1.20958496   1.28878905
   1.37317945   1.46309577   1.55889984   1.6609772    1.76973863
   1.8856218    2.00909304   2.14064922   2.28081976   2.43016872
   2.5892971    2.75884527   2.93949552   3.13197482   3.33705774
   3.55556956   3.78838962   4.03645484   4.30076345   4.5823791
   4.88243505   5.20213877   5.54277682   5.90571998   6.2924288
   6.70445946   7.14347005   7.61122722   8.10961331   8.64063391
   9.20642595   9.80926626  10.45158079  11.13595433  11.8651409
  12.6420749   13.46988283  14.35189595  15.29166362  16.29296764
  17.35983743  18.49656627  19.70772855  20.99819821  22.37316831
  23.83817199  25.39910467  27.06224782  28.83429423  30.72237491
  32.73408785  34.87752853  37.16132253  39.59466023  42.18733383
  44.94977669  47.89310538  51.02916436  54.37057369  57.93077979
  61.72410955  65.76582801  70.07219975  74.66055437  79.54935622
  84.75827869  90.30828341  96.22170458 102.52233885 109.23554107
 116.38832636

# Main Regions of Interest

## Hippocampus

In [9]:
hc_list

['J4-J5',
 'J5-J6',
 'J6-J7',
 'K1-K2',
 'K2-K3',
 'K3-K4',
 'K4-K5',
 'K5-K6',
 'K6-K7',
 'K7-K8']

In [10]:
# only ROI of interest
trial_onset_roi = trial_onset_epochs.copy().pick_channels(hc_list)


In [11]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [12]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)


computing TFR


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:  2.3min remaining:  1.5min
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:  3.2min finished


Not setting metadata


In [13]:
# Crop to time of interest

roi_tfr.crop(tmin = -1, tmax = 5)

<EpochsTFR | time : [-1.000000, 5.000000], freq : [1.000000, 150.000000], epochs : 163, channels : 10, ~5.83 GB>

In [14]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/hc-tfr.h5", overwrite = True)

FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = '/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/SLCH002/ieeg/trial_onset/hc-tfr.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Hippocampal TFR at Trial Onset, n={len(hc_list)}", subject, 'hc_trialonset')


## Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')
    


## Export freq data locked to turn around time to csvs

# OFC

In [ ]:
ofc_list

In [ ]:
# only roi of interest
trial_onset_roi = trial_onset_epochs.copy().pick_channels(ofc_list)


In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/ofc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average OFC TFR at Trial Onset, n={len(ofc_list)}", subject, 'ofc_trialonset')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

# Amygdala

In [ ]:
amyg_list

In [ ]:
# only roi 
trial_onset_roi = trial_onset_epochs.copy().pick_channels(amyg_list)


In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/amyg-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Amygdala TFR at Trial Onset, n={len(amyg_list)}", subject, 'amyg_trialonset')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

## Cingulate

In [ ]:
cing_list

In [ ]:
# only roi 
trial_onset_roi = trial_onset_epochs.copy().pick_channels(cing_list)


In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/cing-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Ant. Cingulate TFR at Trial Onset, n={len(cing_list)}", subject, 'cing_trialonset')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
# Only ROI
trial_onset_roi = trial_onset_epochs.copy().pick_channels(insula_list)


In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/insula-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Insula TFR at Trial Onset, n={len(insula_list)}", subject, 'insula_trialonset')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
# Only ROI
trial_onset_roi = trial_onset_epochs.copy().pick_channels(dlpfc_list)


In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/dlpfc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average dlPFC TFR at Trial Onset, n={len(dlpfc_list)}", subject, 'dlpfc_trialonset')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

## MFG

In [ ]:
mfg_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(mfg_list)
plot_average_tfr(tmp_tfr, f"Average MFG TFR at Trial Onset, n={len(mfg_list)}", subject, 'mfg_trialonset')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in mfg_list:
    chix = dlpfc_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')
    

## SFG

In [ ]:
sfg_list

In [ ]:
# Plot TFR
tmp_tfr = roi_tfr.copy().pick_channels(sfg_list)
plot_average_tfr(tmp_tfr, f"Average SFG TFR at Trial Onset, n={len(sfg_list)}", subject, 'sfg_trialonset')

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in sfg_list:
    chix = dlpfc_list.index(ch)
    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')

## Entorhinal Cortex

In [ ]:
ec_list

In [ ]:
# Only ROI
trial_onset_roi = trial_onset_epochs.copy().pick_channels(ec_list)

In [ ]:
# Resample to 1000 
if trial_onset_roi.info['sfreq'] > 1000:
    trial_onset_roi= trial_onset_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(trial_onset_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 5)

In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/trial_onset/ec-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,5), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Entorhinal TFR at Trial Onset, n={len(ec_list)}", subject, 'ec_trialonset')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Trial Onset')